In [15]:
import os
import yaml
from itertools import product

In [16]:
BASE_CONFIG_PATH = 'configs/movies_config.yaml'

In [3]:
def deep_update(original, update):
    """Deep update original dict with values from the update dict."""
    for key, value in update.items():
        if isinstance(value, dict):
            original[key] = deep_update(original.get(key, {}), value)
        else:
            original[key] = value
    return original

def save_config(config, directory, experiment_name):
    full_path = os.path.join(directory, experiment_name)
    os.makedirs(full_path, exist_ok=True)
    with open(os.path.join(full_path, "config.yaml"), 'w') as f:
        yaml.dump(config, f)


In [4]:
def generate_experiment_name(params):
    # Convert each parameter to a string of the form "paramName_paramValue"
    # and join them all with underscores
    return "_".join([f"{param}_{value}" for param, value in params.items()])

In [5]:
def save_base_config(base_config, experiment_set):
    # Update the master_path and dir keys
    base_config['paths']['eval_results_master_path'] = f"experiments/{experiment_set}/eval_results.csv"
    base_config['paths']['experiment_dir'] = f"experiments/{experiment_set}"
    
    # Write the updated base config to the experiment set directory
    with open(f"experiments/{experiment_set}/config.yaml", 'w') as f:
        yaml.dump(base_config, f)


In [17]:
# Define the mapping from parameter values to specific config changes
value_mapping = {
        'data': {
            'shapes': {'data': {'data_path': 'data/shapes.json'}},
            'movies': {'data': {'data_path': 'data/movielens_16.json'}},
            'clothing': {'data': {'data_path': 'data/clothing.json'}},
            'movies_subset': {'data': {'data_path': 'data/movielens_16_trimmed.json'}},
            'movies_100': {'data': {'data_path': 'data/ml25M_100_movie_sample.json'}},
        },
        'users': {
            'shapes': {'data': {'user_path': 'data/users/shapes_qrel.tsv'}},
            'movies': {'data': {'user_path': 'data/users/movielens_16_qrel.tsv'}},
            'clothing': {'data': {'user_path': 'data/users/clothing_qrel.tsv'}},
            'movies_subset': {'data': {'user_path': 'data/users/movielens_16_qrel_trimmed.tsv'}},
            'movies_5': {'data': {'user_path': 'data/users/movielens_5_users.tsv'}},
        },
        'num_turns': {
            3: {'dialogue_sim': {'num_turns': 3}},
            5: {'dialogue_sim': {'num_turns': 5}},
            10: {'dialogue_sim': {'num_turns': 10}},
            15: {'dialogue_sim': {'num_turns': 15}},
            20: {'dialogue_sim': {'num_turns': 20}}
        },
        'temperature': {
            0: {'llm': {'temperature': 0}},
            1: {'llm': {'temperature': 1}},
        },
        'pe_module': {
            'DT': {'pe': {'pe_module_name': 'DT'}},
            'MonoLLM': {'pe': {'pe_module_name': 'MonoLLM'}},
        },
        'response_update': {
            'individual': {'pe': {'response_update': 'individual'}},
            'concat': {'pe': {'response_update': 'concat'}},
        },
        'num_recs': {
            1: {'pe': {'num_recs': 1}},
            5: {'pe': {'num_recs': 5}},
            10: {'pe': {'num_recs': 10}},
        },
        'item_selection': {
            'greedy': {'query': {'item_selection': 'greedy'}},
            'random': {'query': {'item_selection': 'random'}},
            'entropy_reduction': {'query': {'item_selection': 'entropy_reduction'}},
            'ucb': {'query': {'item_selection': 'ucb'}},
            'thompson': {'query': {'item_selection': 'thompson'}},
        },        
        'preprocessing': {
            'aspect_value': {'item_scoring': {'history_preprocessor_name': 'AspectValuePreprocessor'}},
            'aspect_key_value': {'item_scoring': {'history_preprocessor_name': 'AspectKVPreprocessor'}},
            'concat': {'item_scoring': {'history_preprocessor_name': 'HistoryConcatenator'}},
        },
        'scorer': {
            'mnli': {'item_scoring': {'item_scorer_name': 'MNLIScorer'}},
            'ce': {'item_scoring': {'item_scorer_name': 'CEScorer'}},
        }
}

In [18]:
# Define the parameter grid
param_grid = {
        'data': ['movies_100'],
        'users': ['movies_5'],
        'num_turns': [10],
        'temperature': [0],
        'pe_module': ['DT'],
        'response_update': ['individual'],
        'num_recs': [10],
        'item_selection': ['greedy', 'random', 'entropy_reduction', 'ucb'],
        'preprocessing': ['aspect_key_value'],
        'scorer': ['mnli'],
    }

# NOTE: Current eval_manager can't handle multiple user ground truth files


In [19]:
#experiment set name:
experiment_set = f'movies_100_dt_methods'

In [20]:

# Load the base config file
with open(BASE_CONFIG_PATH, 'r') as f:
    base_config = yaml.safe_load(f)

# Generate and save config files for each combination
for idx, param_values in enumerate(product(*param_grid.values())):
        param_values_dict = dict(zip(param_grid.keys(), param_values))
        #print(param_values_dict)
        experiment_name = generate_experiment_name(param_values_dict)
        updated_config = yaml.safe_load(yaml.dump(base_config))  # deep copy


        # Apply updates to the config based on the current parameter values
        for param, value in param_values_dict.items():
            if param in value_mapping and value in value_mapping[param]:
                deep_update(updated_config, value_mapping[param][value])
            else:
                print(f"No mapping found for parameter '{param}' with value '{value}'")

        save_config(updated_config, f'experiments/{experiment_set}', experiment_name)

#saves base config for evaluation purposes        
save_base_config(base_config, experiment_set)

In [21]:
len(list(product(*param_grid.values())))

4